# Jonathan Halverson
# Monday, November 6, 2017
# Scitkit-Learn MLP versus Tensorflow

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('halverson')

In [3]:
from sklearn.datasets import load_digits

In [9]:
dir(load_digits())

['DESCR', 'data', 'images', 'target', 'target_names']

In [4]:
X, y = load_digits().data, load_digits().target

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [11]:
X_train

array([[  0.,   0.,   1., ...,  12.,   6.,   0.],
       [  0.,   0.,   5., ...,   9.,   0.,   0.],
       [  0.,   0.,   0., ...,  11.,   0.,   0.],
       ..., 
       [  0.,   0.,   8., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,  13.,   1.,   0.],
       [  0.,   0.,   3., ...,   0.,   0.,   0.]])

In [13]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

std_sc = StandardScaler()
X_train_std = std_sc.fit_transform(X_train)
X_test_std = std_sc.transform(X_test)
mm_sc = MinMaxScaler()
X_train_mm = mm_sc.fit_transform(X_train)
X_test_mm = mm_sc.transform(X_test)

In [16]:
X_train_mm.min(), X_train_mm.max()

(0.0, 1.0)

In [26]:
from sklearn.neural_network import MLPClassifier

In [53]:
# setting early_stopping to True made the model worse
mlp_clf = MLPClassifier(activation='relu', batch_size='auto', hidden_layer_sizes=(100, 50, 25),
                        learning_rate='constant', early_stopping=False, alpha=0.0, max_iter=400)
mlp_clf.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=0.0, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 50, 25), learning_rate='constant',
       learning_rate_init=0.001, max_iter=400, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [54]:
from sklearn.metrics import accuracy_score

accuracy_score(mlp_clf.predict(X_train), y_train)

1.0

In [55]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(mlp_clf, X_train, y_train, cv=10)
scores.min(), scores.mean(), scores.max()

(0.94814814814814818, 0.97031951924487969, 0.98561151079136688)